# Notebook to Visualize and Compare Reference Data 

In [20]:
# !pip install pandas_bokeh

In [21]:
import pandas as pd
import numpy as np
import boto3
import io

import pandas_bokeh
from bokeh.io import output_file, show
from bokeh.models import BasicTicker, ColorBar, LinearColorMapper, ColumnDataSource, PrintfTickFormatter
from bokeh.plotting import figure, save
from bokeh.transform import transform
from bokeh.layouts import grid, row


import datetime as dt
from pytz import timezone

import matplotlib.pyplot as plt
%matplotlib inline

from datetime import timedelta



In [22]:
pandas_bokeh.output_notebook()
# output_file(filename="custom_filename.html", title="Static HTML file")

Loading BokehJS ...

## decide to export as html or embedded in notebook

In [23]:
export_html=True # if we want to export html file (True) or show in notebook (false)

In [24]:
# rainpreddf=rainpreddf.reset_index()
# rainpreddf.set_index(pd.DatetimeIndex(rainpreddf.time), inplace=True)

In [25]:
startdate='2022-09-1'
enddate='2022-09-20'

## Process Apogee

In [26]:
def setdftime(df):
    '''set dataframe time to be time index'''
    df=df.reset_index()
    df['TIMESTAMP'] = pd.to_datetime(df.TIMESTAMP,  errors='coerce') 
    #, format='%Y-%m-%d %H:%M:%S' commented out cause wncwxt timestamp error
    df.set_index(pd.DatetimeIndex(df['TIMESTAMP']) , inplace=True)
    return df


In [27]:
APGdf=pd.read_csv('s3://arable-adse-dev/reference_SFRT_TW/SF_Apogee.csv', error_bad_lines=False, skiprows=3)
APGdf.columns=['TIMESTAMP','RECORD','BattV_Avg','PTemp_C_Avg','SWin_Avg','SWout_Avg','LWin_Avg','LWout_Avg','SWnet_Avg','LWnet_Avg','SWalbedo_Avg','NR_Avg']

b'Skipping line 8251: expected 12 fields, saw 16\n'


In [28]:
APGdf=setdftime(APGdf)
# APGdf.set_index('TIMESTAMP', inplace=True)

In [29]:
#assign datatype to numeric
APGdf=APGdf.loc[:, APGdf.columns!='TIMESTAMP'].astype(float)
# APGdf=APGdf[~'TIMESTAMP'].astype(float)

In [30]:
features=['BattV_Avg','PTemp_C_Avg','SWin_Avg','SWout_Avg','LWin_Avg','LWout_Avg','SWnet_Avg','LWnet_Avg','SWalbedo_Avg','NR_Avg']

## Process Apogee_WNC

In [31]:
APGdfWNC=pd.read_csv('s3://arable-adse-dev/reference_SFRT_TW/WNC_Apogee.csv', error_bad_lines=False, skiprows=3)
APGdfWNC.columns=['TIMESTAMP','RECORD','BattV_Avg','PTemp_C_Avg','SWin_Avg','SWout_Avg','LWin_Avg','LWout_Avg','SWnet_Avg','LWnet_Avg','SWalbedo_Avg','NR_Avg']

b'Skipping line 7161: expected 12 fields, saw 22\nSkipping line 8629: expected 12 fields, saw 22\nSkipping line 9347: expected 12 fields, saw 23\nSkipping line 14535: expected 12 fields, saw 17\nSkipping line 14863: expected 12 fields, saw 17\nSkipping line 17501: expected 12 fields, saw 17\n'
b'Skipping line 82140: expected 12 fields, saw 23\nSkipping line 84987: expected 12 fields, saw 22\n'


In [32]:
# APGdfWNC.head()

In [33]:
APGdfWNC=setdftime(APGdfWNC)
# APGdfWNCWNC.set_index('TIMESTAMP', inplace=True)

In [34]:
#assign datatype to numeric
APGdfWNC=APGdfWNC.loc[:, APGdfWNC.columns!='TIMESTAMP'].astype(float)
# APGdfWNC=APGdfWNC[~'TIMESTAMP'].astype(float)

In [35]:
features=['SWin_Avg','SWout_Avg','LWin_Avg','LWout_Avg','SWnet_Avg','LWnet_Avg','SWalbedo_Avg','NR_Avg']

## 1. Plot Apogee_WNC vs. Apogee TimeSeries

In [36]:
ax=[0]*len(features)
for i,feature in enumerate(features):
    ax[i] = figure(width=800, height=200, x_axis_type="datetime" ) 
    ax[i].line(APGdf.loc[startdate: enddate].index.values, APGdf.loc[startdate: enddate][feature], line_color='red', \
           legend_label="SF_APG")
    ax[i].line(APGdfWNC.loc[startdate: enddate].index.values, APGdfWNC.loc[startdate: enddate][feature], line_color='green', \
           legend_label="APGWNC")
    ax[i].title.text = f'{feature} '

#     show(p)

res=[ax[i] for i in range(8)]
grid_layout = grid(res, nrows=4)

if export_html:
    output_file("APG_compare_timeseries.html")
    save(grid_layout)
else: 
    show(grid_layout)


In [37]:
#drop null value in TIMESTAMP

APGdf = APGdf[APGdf.index.notnull()]
APGdfWNC = APGdfWNC[APGdfWNC.index.notnull()]
APGdfWNC=APGdfWNC.sort_index()
APGdf=APGdf.sort_index()

In [38]:
## xplots
MergeAPGdfnAPGdfWXT=pd.merge_asof(APGdf, APGdfWNC, left_on='TIMESTAMP', right_on='TIMESTAMP', by='TIMESTAMP')

In [39]:
print(MergeAPGdfnAPGdfWXT.columns.tolist())

['TIMESTAMP', 'index_x', 'RECORD_x', 'BattV_Avg_x', 'PTemp_C_Avg_x', 'SWin_Avg_x', 'SWout_Avg_x', 'LWin_Avg_x', 'LWout_Avg_x', 'SWnet_Avg_x', 'LWnet_Avg_x', 'SWalbedo_Avg_x', 'NR_Avg_x', 'index_y', 'RECORD_y', 'BattV_Avg_y', 'PTemp_C_Avg_y', 'SWin_Avg_y', 'SWout_Avg_y', 'LWin_Avg_y', 'LWout_Avg_y', 'SWnet_Avg_y', 'LWnet_Avg_y', 'SWalbedo_Avg_y', 'NR_Avg_y']


In [40]:
features3= ['SWin_Avg', 'SWout_Avg', 'LWin_Avg', 'LWout_Avg', 
            'SWnet_Avg', 'LWnet_Avg', 'SWalbedo_Avg', 'NR_Avg']
# MergeAPGdfnAPGdfWXT

## 2. Plot Apogee_WNC vs. Apogee Cross Plot

In [41]:
p=[0]*len(features3)
    
for i, feature in enumerate(features3):
    p[i] = figure(width=300, height=300) 
#     f, p[i] = plt.subplots(figsize=(6, 6))
    p[i].circle(x=eval(f"MergeAPGdfnAPGdfWXT.{feature}_x"),y=eval(f"MergeAPGdfnAPGdfWXT.{feature}_y"))
    
#     p[i] = MergeAPGdfnAPGdfWXT.plot_bokeh(kind="scatter", x=eval(f'{feature}_x'),y=eval(f'{feature}_y'),show_figure=False)
    p[i].title.text = f'{feature} '

#     show(p[i])
res=[p[i] for i in range(8)]
grid_layout = grid(res, nrows=3)

if export_html:
    output_file("APG_compare_xplots.html")
    save(grid_layout)
else: 
    show(grid_layout)
    

# Process WXT Data

In [42]:
WNCWXT=pd.read_csv('s3://arable-adse-dev/reference_SFRT_TW/WNC_WXT536.csv', error_bad_lines=False, skiprows=3)
WNCWXT.columns=['TIMESTAMP','RECORD','WindDir_wxt','WS_ms_wxt_Avg','WS_ms_wxt_Max','WS_ms_wxt_TMx','WS_ms_wxt_Min','WS_ms_wxt_TMn','AirTC_wxt_Avg','RH_wxt_Avg','BP_wxt_Avg','Rain_mm_wxt_Avg','Rain_mm_wxt_Tot']

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3258: DtypeWarning: Columns (2,3,4,6,8,9,10,11,12) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [43]:

setdftime(WNCWXT)
WNCWXT.head(2)


,TIMESTAMP,RECORD,WindDir_wxt,WS_ms_wxt_Avg,WS_ms_wxt_Max,WS_ms_wxt_TMx,WS_ms_wxt_Min,WS_ms_wxt_TMn,AirTC_wxt_Avg,RH_wxt_Avg,BP_wxt_Avg,Rain_mm_wxt_Avg,Rain_mm_wxt_Tot
0,7/14/22 20:27,54,267,1.8,3.4,7/14/22 20:26,0.6,7/14/22 20:26,19.45,61.2,101.2,0,0
1,7/14/22 20:28,55,110,1.35,3.2,7/14/22 20:27,0.5,7/14/22 20:27,19.35,61.7,101.2,0,0


## Process WXT

In [44]:
# WXT=pd.read_csv('s3://arable-adse-dev/rain_classification_april_2022/SFRT_TW_ref/WNC_WXT536.csv', error_bad_lines=False, skiprows=4)
WXT=pd.read_csv('s3://arable-adse-dev/reference_SFRT_TW/SF_WXT536.csv', error_bad_lines=False, skiprows=4)
WXT.columns=['TIMESTAMP','RECORD','WindDir_wxt','WS_ms_wxt_Avg','WS_ms_wxt_Max','WS_ms_wxt_TMx','WS_ms_wxt_Min','WS_ms_wxt_TMn','AirTC_wxt_Avg','RH_wxt_Avg','BP_wxt_Avg','Rain_mm_wxt_Avg','Rain_mm_wxt_Tot']

In [45]:
WXT=setdftime(WXT)
# WXT

In [46]:
#assign datatype to numeric
WXT=WXT[WXT.columns[~WXT.columns.isin(['TIMESTAMP', 'WS_ms_wxt_TMx', 'WS_ms_wxt_TMn'])]].astype(float)

In [47]:
features=['WindDir_wxt','WS_ms_wxt_Avg','WS_ms_wxt_Max','WS_ms_wxt_Min','AirTC_wxt_Avg','RH_wxt_Avg','BP_wxt_Avg','Rain_mm_wxt_Avg','Rain_mm_wxt_Tot']

## Process WXTWNC

In [48]:
WXTWNC=pd.read_csv('s3://arable-adse-dev/reference_SFRT_TW/WNC_WXT536.csv', error_bad_lines=False, skiprows=4)
WXTWNC.columns=['TIMESTAMP','RECORD','WindDir_wxt','WS_ms_wxt_Avg','WS_ms_wxt_Max','WS_ms_wxt_TMx','WS_ms_wxt_Min','WS_ms_wxt_TMn','AirTC_wxt_Avg','RH_wxt_Avg','BP_wxt_Avg','Rain_mm_wxt_Avg','Rain_mm_wxt_Tot']

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3258: DtypeWarning: Columns (2,3,4,6,8,9,10,11,12) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [49]:
WXTWNC=setdftime(WXTWNC)
# APGdfWNCWNC.set_index('TIMESTAMP', inplace=True)

In [50]:
#assign datatype to numeric
WXTWNC=WXTWNC[WXTWNC.columns[~WXTWNC.columns.isin(['TIMESTAMP', 'WS_ms_wxt_TMx', 'WS_ms_wxt_TMn'])]].astype(float)

In [51]:
features2=['WindDir_wxt', 'WS_ms_wxt_Avg', 'WS_ms_wxt_Max', 
           'WS_ms_wxt_Min', 'AirTC_wxt_Avg', 'RH_wxt_Avg', 'BP_wxt_Avg', 'Rain_mm_wxt_Avg', 'Rain_mm_wxt_Tot']

## 3. Plot WXT vs. WXTNC TimeSeries

In [52]:
p=[0]*len(features2)
for i, feature in enumerate(features2):
    p[i] = figure(width=800, height=200,x_axis_type="datetime") 
    p[i].line(WXT.loc[startdate: enddate].index.values, WXT.loc[startdate: enddate][feature], line_color='red', \
           legend_label="SF_WXT")
    p[i].line(WXTWNC.loc[startdate: enddate].index.values, WXTWNC.loc[startdate: enddate][feature], line_color='green', \
           legend_label="WXTWNC")
    p[i].title.text = f'{feature} '

#     show(p)

res=[p[i] for i in range(len(features2))]
grid_layout = grid(res, nrows=5)

if export_html:
    output_file("WXT_compare_timeseries.html")
    save(grid_layout)
else: 
    show(grid_layout)

In [53]:
#drop null value in TIMESTAMP

WXT = WXT[WXT.index.notnull()]
WXTWNC = WXTWNC[WXTWNC.index.notnull()]
WXTWNC=WXTWNC.sort_index()
WXT=WXT.sort_index()

In [54]:
## xplots
MergeWXTnWXTWNC=pd.merge_asof(WXT, WXTWNC, left_on='TIMESTAMP', right_on='TIMESTAMP', by='TIMESTAMP')

In [55]:
feature2=[c for c in MergeWXTnWXTWNC.columns.tolist() if c not in ['TIMESTAMP', 'index_x', 'index_y' , 'RECORD_x', 'RECORD_y']]

In [56]:
print(MergeWXTnWXTWNC.columns.tolist())

['TIMESTAMP', 'index_x', 'RECORD_x', 'WindDir_wxt_x', 'WS_ms_wxt_Avg_x', 'WS_ms_wxt_Max_x', 'WS_ms_wxt_Min_x', 'AirTC_wxt_Avg_x', 'RH_wxt_Avg_x', 'BP_wxt_Avg_x', 'Rain_mm_wxt_Avg_x', 'Rain_mm_wxt_Tot_x', 'index_y', 'RECORD_y', 'WindDir_wxt_y', 'WS_ms_wxt_Avg_y', 'WS_ms_wxt_Max_y', 'WS_ms_wxt_Min_y', 'AirTC_wxt_Avg_y', 'RH_wxt_Avg_y', 'BP_wxt_Avg_y', 'Rain_mm_wxt_Avg_y', 'Rain_mm_wxt_Tot_y']


## 4. Plot WXT vs. WXTNC Crossplots

In [57]:
feature2=['WindDir_wxt', 'WS_ms_wxt_Avg', 'WS_ms_wxt_Max', 'WS_ms_wxt_Min', 'AirTC_wxt_Avg', 'RH_wxt_Avg', 'BP_wxt_Avg', 'Rain_mm_wxt_Avg', 'Rain_mm_wxt_Tot']
ay=[0]*len(features2)
for i, feature in enumerate(features2):
    ay[i]= figure(width=300, height=300)
    ay[i].circle(x=eval(f'MergeWXTnWXTWNC.{feature}_x'), y=eval(f'MergeWXTnWXTWNC.{feature}_y'))
#     p.line(x=[eval(f'MergeWXTnWXTWNC.{feature}_x.min()'), eval(f'MergeWXTnWXTWNC.{feature}_y.min()')],
#            y=[eval(f'MergeWXTnWXTWNC.{feature}_x.may()'), eval(f'MergeWXTnWXTWNC.{feature}_y.may()')])
    ay[i].title.text = f'{feature}'

#     show(ay[i])
res=[ay[i] for i in range(len(feature2))]
grid_layout = grid(res, nrows=4)

if export_html:
    output_file("WXT_compare_xplots.html")
    save(grid_layout)
else: 
    show(grid_layout)

## Process Spectrafy

In [58]:
SF_Spetrafy=pd.read_csv('s3://arable-adse-dev/reference_SFRT_TW/SF_Spectrafy_G_dw.csv', error_bad_lines=False, skiprows=3) #, skiprows=4
SF_Spetrafy.columns=["TIMESTAMP","RECORD","AmbientTemperature_dw_Avg","AmbientPressure_dw_Avg","AmbientHumidity_dw_Avg","InternalTemp_dw_Avg","InternalHumidity_dw_Avg","ChannelVolt1_dw_Avg","ChannelVolt2_dw_Avg","ChannelVolt3_dw_Avg","ChannelVolt4_dw_Avg","ChannelVolt5_dw_Avg","ChannelVolt6_dw_Avg","ChannelVolt7_dw_Avg","ChannelVolt8_dw_Avg","ChannelVolt9_dw_Avg"]

b'Skipping line 11726: expected 16 fields, saw 24\nSkipping line 13007: expected 16 fields, saw 27\nSkipping line 13422: expected 16 fields, saw 25\n'


In [59]:
SF_Spetrafy.sample(5)

,TIMESTAMP,RECORD,AmbientTemperature_dw_Avg,AmbientPressure_dw_Avg,AmbientHumidity_dw_Avg,InternalTemp_dw_Avg,InternalHumidity_dw_Avg,ChannelVolt1_dw_Avg,ChannelVolt2_dw_Avg,ChannelVolt3_dw_Avg,ChannelVolt4_dw_Avg,ChannelVolt5_dw_Avg,ChannelVolt6_dw_Avg,ChannelVolt7_dw_Avg,ChannelVolt8_dw_Avg,ChannelVolt9_dw_Avg
13731,2022-09-09 14:21:00,13480,14.48889,99.94538,70.33495,16.1862,3.776753,95.63766,92.68433,81.29183,64.45583,92.46582,64.131,28.11283,58.738,7.211833
5051,2022-09-03 12:54:00,4782,14.30126,101.1304,71.7819,16.16016,3.749185,0.181,0.2698333,0.2116667,0.2616667,0.1858333,0.1791667,0.2218333,0.2515,0.281
14031,2022-09-09 19:29:00,13780,29.82499,99.82428,28.86847,31.84244,3.979536,1867.393,1765.737,1641.458,1668.552,2177.802,1790.151,1392.226,2376.296,304.7305
8361,2022-09-05 20:25:00,8092,42.8547,100.4857,13.53138,44.61719,4.17908,2014.126,1866.066,1726.524,1745.762,2271.242,1857.333,1205.329,2428.221,324.1591
11502,2022-09-08 00:47:00,11233,26.17089,100.9635,42.30587,28.36458,3.933978,543.4314,552.0788,550.4023,572.4885,795.3108,685.1888,223.6493,903.3091,118.6987


In [60]:
SF_Spetrafy=setdftime(SF_Spetrafy)
# APGdfWNCWNC.set_index('TIMESTAMP', inplace=True)

In [61]:
#assign datatype to numeric
SF_Spetrafy=SF_Spetrafy[SF_Spetrafy.columns[~SF_Spetrafy.columns.isin(['TIMESTAMP', 'RECORD'])]].astype(float)

In [62]:
features=["AmbientTemperature_dw_Avg","AmbientPressure_dw_Avg","AmbientHumidity_dw_Avg","InternalTemp_dw_Avg","InternalHumidity_dw_Avg","ChannelVolt1_dw_Avg","ChannelVolt2_dw_Avg","ChannelVolt3_dw_Avg","ChannelVolt4_dw_Avg","ChannelVolt5_dw_Avg","ChannelVolt6_dw_Avg","ChannelVolt7_dw_Avg","ChannelVolt8_dw_Avg","ChannelVolt9_dw_Avg"]

In [63]:
WNC_Spetrafy=pd.read_csv('s3://arable-adse-dev/reference_SFRT_TW/WNC_Spectrafy_G_dw.csv', error_bad_lines=False, skiprows=3) #, skiprows=4
WNC_Spetrafy.columns=["TIMESTAMP","RECORD","AmbientTemperature_dw_Avg","AmbientPressure_dw_Avg","AmbientHumidity_dw_Avg","InternalTemp_dw_Avg","InternalHumidity_dw_Avg","ChannelVolt1_dw_Avg","ChannelVolt2_dw_Avg","ChannelVolt3_dw_Avg","ChannelVolt4_dw_Avg","ChannelVolt5_dw_Avg","ChannelVolt6_dw_Avg","ChannelVolt7_dw_Avg","ChannelVolt8_dw_Avg","ChannelVolt9_dw_Avg"]

b'Skipping line 20069: expected 16 fields, saw 23\n'
b'Skipping line 40177: expected 16 fields, saw 24\nSkipping line 41072: expected 16 fields, saw 25\nSkipping line 45327: expected 16 fields, saw 28\n'
/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3258: DtypeWarning: Columns (2,3,4,5,6,7,8,9,10,11,12,13,14,15) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [64]:
WNC_Spetrafy=setdftime(WNC_Spetrafy)
# APGdfWNCWNC.set_index('TIMESTAMP', inplace=True)

In [65]:
#assign datatype to numeric
WNC_Spetrafy=WNC_Spetrafy[WNC_Spetrafy.columns[~WNC_Spetrafy.columns.isin(['TIMESTAMP', 'RECORD'])]].astype(float)

In [66]:
features=["AmbientTemperature_dw_Avg","AmbientPressure_dw_Avg","AmbientHumidity_dw_Avg","InternalTemp_dw_Avg","ChannelVolt1_dw_Avg","ChannelVolt2_dw_Avg","ChannelVolt3_dw_Avg","ChannelVolt4_dw_Avg","ChannelVolt5_dw_Avg","ChannelVolt6_dw_Avg","ChannelVolt7_dw_Avg","ChannelVolt8_dw_Avg","ChannelVolt9_dw_Avg"]

## 3. Plot SF vs. WNC Spectrafy TimeSeries

In [67]:
p=[0]*len(features)
for i, feature in enumerate(features):
    p[i] = figure(width=800, height=200,x_axis_type="datetime") 
    p[i].line(SF_Spetrafy.loc[startdate: enddate].index.values, SF_Spetrafy.loc[startdate: enddate][feature], line_color='red', \
           legend_label="SF_Spetrafy")
    p[i].line(WNC_Spetrafy.loc[startdate: enddate].index.values, WNC_Spetrafy.loc[startdate: enddate][feature], line_color='green', \
           legend_label="WNC_Spetrafy")
    p[i].title.text = f'{feature} '

res=[p[i] for i in range(9)]
grid_layout = grid(res, nrows=5)

if export_html:
    output_file("Spectrafy_compare_timeseries.html")
    save(grid_layout)
else: 
    show(grid_layout)

In [68]:
#drop null value in TIMESTAMP

WNC_Spetrafy = WNC_Spetrafy[WNC_Spetrafy.index.notnull()]
SF_Spetrafy = SF_Spetrafy[SF_Spetrafy.index.notnull()]
SF_Spetrafy=SF_Spetrafy.sort_index()
WNC_Spetrafy=WNC_Spetrafy.sort_index()

In [69]:
## xplots
MergeWNC_SF_Spetrafy=pd.merge_asof(WNC_Spetrafy, SF_Spetrafy, left_on='TIMESTAMP', right_on='TIMESTAMP', by='TIMESTAMP')

In [70]:
feature2=[c for c in MergeWNC_SF_Spetrafy.columns.tolist() if c not in ['TIMESTAMP', 'index_x', 'index_y' , 'RECORD_x', 'RECORD_y']]

In [71]:
print(MergeWNC_SF_Spetrafy.columns.tolist())

['TIMESTAMP', 'index_x', 'AmbientTemperature_dw_Avg_x', 'AmbientPressure_dw_Avg_x', 'AmbientHumidity_dw_Avg_x', 'InternalTemp_dw_Avg_x', 'InternalHumidity_dw_Avg_x', 'ChannelVolt1_dw_Avg_x', 'ChannelVolt2_dw_Avg_x', 'ChannelVolt3_dw_Avg_x', 'ChannelVolt4_dw_Avg_x', 'ChannelVolt5_dw_Avg_x', 'ChannelVolt6_dw_Avg_x', 'ChannelVolt7_dw_Avg_x', 'ChannelVolt8_dw_Avg_x', 'ChannelVolt9_dw_Avg_x', 'index_y', 'AmbientTemperature_dw_Avg_y', 'AmbientPressure_dw_Avg_y', 'AmbientHumidity_dw_Avg_y', 'InternalTemp_dw_Avg_y', 'InternalHumidity_dw_Avg_y', 'ChannelVolt1_dw_Avg_y', 'ChannelVolt2_dw_Avg_y', 'ChannelVolt3_dw_Avg_y', 'ChannelVolt4_dw_Avg_y', 'ChannelVolt5_dw_Avg_y', 'ChannelVolt6_dw_Avg_y', 'ChannelVolt7_dw_Avg_y', 'ChannelVolt8_dw_Avg_y', 'ChannelVolt9_dw_Avg_y']


## 4. Plot WNC_Spetrafy vs. WNC_SpetrafyNC Crossplots

In [72]:
feature2=["AmbientTemperature_dw_Avg","AmbientPressure_dw_Avg","AmbientHumidity_dw_Avg","ChannelVolt1_dw_Avg","ChannelVolt2_dw_Avg","ChannelVolt3_dw_Avg","ChannelVolt4_dw_Avg","ChannelVolt5_dw_Avg","ChannelVolt6_dw_Avg","ChannelVolt7_dw_Avg","ChannelVolt8_dw_Avg","ChannelVolt9_dw_Avg"]
ay=[0]*len(feature2)
for i, feature in enumerate(feature2):
#     print(feature)
    ay[i]= figure(width=300, height=300)
    ay[i].circle(x=eval(f'MergeWNC_SF_Spetrafy.{feature}_x'), y=eval(f'MergeWNC_SF_Spetrafy.{feature}_y'))
    ay[i].title.text = f'{feature}'

res=[ay[i] for i in range(8)]
grid_layout = grid(res, nrows=3)

if export_html:
    output_file("Spectrafy_compare_xplots.html")
    save(grid_layout)
else: 
    show(grid_layout)

# Summary

## This Notebook could be used to check consistency between WNC and SF reference devices